# QE Scheme for Heston model 

<font color=black size=3 face=times>**Main references**: <br>
         Andersen, L. (2008). Simple and efficient simulation of the Heston stochastic volatility model. *The Journal of Computational Finance, 11*(3), 1–42. https://doi.org/10.21314/JCF.2008.189 <br>
        Van Haastrecht, A., & Pelsser, A. (2010). Efficient, almost exact simulation of the heston stochastic volatility model. *International Journal of Theoretical and Applied Finance, 13*(01), 1–43. https://doi.org/10.1142/S0219024910005668 <br>

<font color=black size=3 face=times>**1.1. Notation declaration**<br>
- $W_X$ and $W_V$ are scalar Brownian motions in some probability measure and assume that $dW_X(t)\dot dW_Y(t)=\rho dt$
- $\widehat{X}$ and $\widehat{V}$ denote discrete-time approximations to $X$ and $V$, i.e. The basic *Euler Scheme would take the form:
 \begin{align*}
    ln\widehat{X}(t+\Delta)&=ln\widehat{X}(t)-\frac{1}{2}\widehat{V}(t)\Delta+\sqrt{\widehat{V}(t)}Z_X\sqrt{\Delta} ,\\
    ln\widehat{V}(t+\Delta)&=\widehat{V}(t)+\kappa(\theta-\widehat{V}(t))\Delta+\epsilon\sqrt{\widehat{V}(t)}Z_V\sqrt{\Delta},
    \end{align*}  
- $V(t),V(t+\Delta)$ are sampled directly from the known conditional distribution of $V(t)$ and $lnX(t),lnX(t+\Delta)$ are from Gaussian distribution. 

<font color=black size=3 face=times>**1.2. General discrete-time approximation schemes**<br>
    &emsp;&emsp;<font color=black size=3 face=times>**Kahl-Jackel Scheme**<br>
    &emsp;&emsp;Using Milstein scheme to discretize the V-process and "IJK" discretization for the stock process:
    \begin{align*}
    ln\widehat{X}(t+\Delta)&=ln\widehat{X}(t)-\frac{\Delta}{4}(\widehat{V}(t+\Delta)+\widehat{V}(t))+\rho\sqrt{\widehat{V}(t)}Z_V\sqrt{\Delta}+\frac{1}{2}\left(\sqrt{\widehat{V}(t+\Delta)}+\sqrt{\widehat{V}(t)}\right)(Z_X\sqrt{\Delta}-\rho Z_V\sqrt{\Delta})+\frac{1}{4}\epsilon\rho\Delta(Z_V^2-1),\tag{$1.1$}\\
    \widehat{V}(t+\Delta) &= \frac{\widehat{V}(t)+\kappa\theta\Delta+\epsilon\sqrt{\widehat{V}(t)}Z_V\sqrt{\Delta}+\frac{1}{4}\epsilon^2\Delta(Z_V^2-1)}{1+\kappa\Delta}, \tag{$1.2$}
    \end{align*}
    <br/>
    &emsp;&emsp;<font color=black size=3 face=times>**Broadie-Kaya Scheme**<br>
\begin{align*}
    V(t+\Delta)&=V(t)+\int_{t}^{t+\Delta}\kappa(\theta-V(u))du+\epsilon \int_{t}^{t+\Delta}\sqrt{V(u)}dW_V(u) ,\tag{$1.3$}\\
    lnX(t+\Delta)&=lnX(t)+\frac{\rho}{\epsilon}(V(t+\Delta)-V(t)-\kappa\theta\Delta)+(\frac{\kappa\rho}{\epsilon}-\frac{1}{2})\int_{t}^{t+\Delta}V(u)du+\sqrt{1-\rho^2}\int_{t}^{t+\Delta}\sqrt{V(u)}dW(u), \tag{$1.4$}
    \end{align*}

<font color=black size=3 face=times>**3.3. QE Scheme**<br>
    &emsp;&emsp; For the non-central chi-square distribution approaches a Gaussian distribution as the non-centrality paremeter approaches $\infty$, but for small $V(t)$, the non-centrality parameter approaches zero, and the Gaussian variable is typically not accurate.
    Let:
    \begin{align*}
    m&=\theta+(\widehat{V}(t)-\theta)e^{-\kappa\Delta},\tag{$1.5$}\\
    s^2&=\frac{\widehat{V}(t)\epsilon^2 e^{-\kappa\Delta}}{\kappa}(1-e^{-\kappa\Delta})+\frac{\theta\epsilon^2}{2\kappa}(1-e^{-\kappa\Delta})^2,\tag{$1.6$}\\
    \Psi&\equiv s^2/m^2 =\frac{\frac{\widehat{V}(t)\epsilon^2 e^{-\kappa\Delta}}{\kappa}(1-e^{-\kappa\Delta})+\frac{\theta\epsilon^2}{2\kappa}(1-e^{-\kappa\Delta})^2}{(\theta+(\widehat{V}(t)-\theta)e^{-\kappa\Delta})^2}, \tag{$1.7$}
    \end{align*}
    &emsp;&emsp; The scheme varies when $\Psi$ is big and when $\Psi$ is small.<br>
    &emsp;&emsp; Assume that some arbitrary level $\Psi_c\in [1,2]$ has beem selected. The detailed algorithm for the QE simulation step from $\widehat{V}(t)$ to $\widehat{V}(t+\Delta)$ is then:<br>
    &emsp;&emsp;1. Given $\widehat{V}(t)$, compute $m$ and $s^2$ from the equations (1.5) and (1.6).<br>
    &emsp;&emsp;2. Compute $\Psi=s^2/m^2$<br>
    &emsp;&emsp;3. Draw a uniform random number $U_V$<br>
    &emsp;&emsp;4. **If** $\Psi\leq \Psi_c$:<br>
    &emsp;&emsp;&emsp;&emsp;(a) Compute a and b as:
    \begin{align*}
    &b^2=2\Psi^{-1}-1+\sqrt{2\Psi^{-1}}\sqrt{{2\Psi^{-1}-1}}\ge 0,\tag{$1.8$}\\
    &a=\frac{m}{1+b^2},\tag{$1.9$}\\
    \end{align*}
    &emsp;&emsp;&emsp;&emsp;(b) Compute $Z_V=\Phi^{-1}(U_V)$<br>
    &emsp;&emsp;&emsp;&emsp;(c) Set $\widehat{V}(t+\Delta)=a(b+Z_V)^2$<br>
    &emsp;&emsp;5.**Otherwise**, if $\Psi>\Psi_c$:<br>
    &emsp;&emsp;&emsp;&emsp;(a) Compute $\beta$ and $p$ as:
    \begin{align*}
    &p=\frac{\Psi-1}{\Psi+1}\in [0,1),\tag{$1.10$}\\
    &\beta=\frac{1-p}{m}=\frac{2}{m(\Psi+1)}>0,\tag{$1.11$}\\
    \end{align*}
    &emsp;&emsp;&emsp;&emsp;(b) Set $\widehat{V}(t+\Delta)=\Psi^{-1}(U_V;p,\beta)$, where $\Psi^{-1}$ is given as:
    $$\Psi^{-1}(u)=\Psi^{-1}(u;p,\beta)=\left\{
\begin{aligned}
&0,  & 0\le u\le p \\
&\beta^{-1}ln(\frac{1-p}{1-u}),& p<u\le1.
\end{aligned},\tag{$1.12$}
\right.
$$
<br/>
    &emsp;&emsp; After simulating the path, calculate:
\begin{align*}
     E\left(S_T|V(T),\int V(t)dt\right)&=S_0 exp\left(\frac{\rho}{\nu}(V(T)-V(0))-\kappa(\theta-\int_{0}^{T}V(t)dt)\right),\tag{$1.13$}\\
     \sigma&=\left((1-\rho^2)\int_{0}^{T}V(t)dt\right)^{\frac{1}{2}},\tag{$1.14$}\\
    \end{align*}
    &emsp;&emsp; Then bring the Forward price and volatility into the option pricing model.

# QE Scheme for 3/2 model 

<font color=black size=3 face=times>**2.1.Derive 3/2 model from heston model**: <br>
    The stock price and volatility can be written as:
    \begin{align*}
    &dv_t=\kappa v_t(\theta-v_t)dt+\nu v_t^{3/2}dZ_t,\tag{$2.1$}\\
    &\frac{dS_t}{S_t}=\sigma_t dW_t=\sigma_t(\rho dZ_t+\rho_{*}dX_t),\tag{$2.2$}\\
    \end{align*}
where $\rho_{*}=\sqrt{1-\rho^2},v_t=\sigma_t^2$<br>
    Changing variable $x_t=1/v_t$, then we have:
    $$dx_t=-\frac{dv_t}{v_t^2}+\frac{(dv_t)^2}{v_t^3}=(\kappa+\nu^2-\kappa\theta x_t)dt-\nu \sqrt{x_t}dZ_t,\tag{$2.3$}$$
    Then it's equal to Heston model with new parameters:
    $$\nu^{'}=-\nu,\quad\kappa^{'}=\kappa \theta,\quad\text{and} \quad\theta^{'}=(\kappa+\nu^2)/(\kappa\theta)$$
    We can calculate $S_T$ by:
   \begin{align*}
    dln(X_t)&=\left(\frac{\kappa+\nu^2/2}{x_t}-\kappa\theta\right)dt-\frac{\nu}{\sqrt{x_t}}dZ_t,\tag{$2.4$}\\
    \int_{0}^{T}\frac{1}{\sqrt{x_t}}dZ_t&=\frac{1}{\nu}\left(log\left(\frac{x_0}{x_T}\right)+(\kappa+\frac{\nu^2}{2})V_T-\kappa\theta T\right),\tag{$2.5$}\\
    log\left(\frac{S_T}{S_0}\right)&=\frac{\rho}{\nu}-\kappa(T\theta-\left(1+\frac{\nu^2}{2\kappa}V_T\right))-\frac{1}{2}V_T+\rho_{*}\sqrt{V_T}U
    \end{align*} 
    Where $V_T=\int_{0}^{T}v_tdt$ and $U \sim N(0,1)$
    \begin{align*}
    E\left(S_T|v_T,V_T\right)&=S_0exp\left(\frac{\rho}{\nu}\left(log(\frac{V_T}{V_0})-\kappa\left(T\theta-(1+\frac{\nu^2}{2\kappa})V_T)\right)\right)-\frac{1}{2}\rho^2V_T\right),\tag{$2.6$}\\
    \sigma&=\left((1-\rho^2)V_T\right)^{\frac{1}{2}},\tag{$2.7$}
    \end{align*} 